In [1]:
import numpy as np
import pandas as pd
import re

from collections import Counter
import string

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter

import os

from tqdm import tqdm

from gensim.models import Word2Vec
import random

import time

import threading

In [2]:
kkma = Kkma()
twitter = Twitter()

C:\Users\Samsung\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [14]:
data_ = pd.read_excel(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\dic_data\드라이기_20201207.xlsx",sheet_name="data_")

In [15]:
data_ = data_.rename(columns={"Unnamed: 0":"main_attr","Product Attribute 측면 속성":"sub_attr","유사 키워드(attr_sim)":"attr_sim",
                              "토큰화 방식으로 각 속성을 나타내는 키워드 조합":"attr_combi"
                      ,"긍정":"POS","부정":"NEG"})

In [16]:
data_.head()

,main_attr,sub_attr,attr_sim,attr_combi,count,POS,NEG
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이 ...","단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐인..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아여...,괴물 토네이도 이런 표현은좀 오바네요
3,온도,냉풍,#차가운 바람 #찬바람,"['냉풍', '차가운 바람', '찬바람', '바람 차갑다']",2476,"가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요, ...","냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이 별로..."
4,온도,온풍,#따뜻한 바람 #부드러운 바람,"['온풍', '따뜻하다 바람', '부드럽다 바람']",2529,"파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 바람 적...","온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로 좋아..."


In [18]:
data_.loc[0,"NEG"]

'바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩어지는 느낌이예요, 바람쐬기가 약해요, 바람강도가 조금 약한듯, 바람세기가 나무늘보저리가라네요, 약바람에서도 바람이 너무 세고, 바람은 쎄지는 않아요, 바람이 세기가 강하지 않지만, 바람이너무쎄다못해 손이덜덜덜, 바람세기가 심한편입니다, 가끔 바람이 약하게 푸슉 나오는 경우가 있는데, 조금만 센 바람이 나왔더라면, 바람 강도가 생각보다 약한거 같습니다, 바람이 어쩜 그렇게도 약할까요, 바람세기는 0에 가까운데 , 바람세기도 그냥 그러네요, 바람세기는 별루네요, 바람은 그닥 강한것 같지 않아요, 강한바람을 원하시는 분들은 비추합니다, 바람세기는 소리를 못 따라가네요, 바람이 넘 약해서 말리는 시간이 길어지네요, 바람세기 강풍이 생각보다 약하네요, 바람세기가 전문가 용은 아닌듯, 약한 바람은 말리는건가 싶을 정도인데, 바람 힘이 더 셌으면 좋긴하겠 는데, 너무 뜨거울정도로 바람이 뜨겁고 세네요, 빌빌거리는 바람, 바람 세기가 시원찮은데'

In [24]:
test_ = list(map(lambda x:x.strip(),data_.loc[0,"POS"].split(",")))

In [32]:
def split_contents(df):
    Pos_ = []
    Neg_ = []
    for i in range(len(df.index)):
        try:
            pos_ = list(map(lambda x:x.strip(),data_.loc[i,"POS"].split(",")))
            neg_ = list(map(lambda x:x.strip(),data_.loc[i,"NEG"].split(",")))
        except:
            pos_,neg_ = [],[]
        Pos_.append(pos_)
        Neg_.append(neg_)
    df["split_pos"] = Pos_
    df["split_neg"] = Neg_
    df = df.drop(columns=["POS","NEG"])
    return df

In [33]:
data_s = split_contents(data_)

In [35]:
data_s.head()

,main_attr,sub_attr,attr_sim,attr_combi,count,split_pos,split_neg
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"[바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","[바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 ..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"[바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이...","[단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,[바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아...,[괴물 토네이도 이런 표현은좀 오바네요]
3,온도,냉풍,#차가운 바람 #찬바람,"['냉풍', '차가운 바람', '찬바람', '바람 차갑다']",2476,"[가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요, ...","[냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이 별..."
4,온도,온풍,#따뜻한 바람 #부드러운 바람,"['온풍', '따뜻하다 바람', '부드럽다 바람']",2529,"[파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 바람 적...","[온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로 좋..."


In [41]:
pos_len = data_s["split_pos"].map(lambda x:len(x))
neg_len = data_s["split_neg"].map(lambda x:len(x))
print("max")
pos_len.max(),pos_len.min(),pos_len.mean()

max


(31, 0, 5.171428571428572)

In [40]:
neg_len.max(),neg_len.min(),neg_len.mean()

(30, 0, 2.4714285714285715)

# pos tagging 

In [50]:
df = data_s
target_ = "pos"
def pos_tagging(df,target_):
    target_col = "split_{}".format(target_)
    tag_1 = []
    for i in range(len(data_s)):
        text_ = data_s.loc[i,target_col]
        tag_2 = []
        for t in text_:
            tag_ = twitter.pos(t)
            tag_2.append(tag_)
        tag_1.append(tag_2)
    df["tag_{}".format(target_)] = tag_1
    return df

In [52]:
data_pos = pos_tagging(data_s,"pos")
data_pos = pos_tagging(data_pos,"neg")

In [54]:
data_pos.head()

,main_attr,sub_attr,attr_sim,attr_combi,count,split_pos,split_neg,tag_pos,tag_neg
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"[바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","[바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 ...","[[(바람, Noun), (도, Josa), (쎄고, Adjective)], [(드...","[[(바람, Noun), (이, Josa), (약해요, Adjective)], [(..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"[바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이...","[단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐...","[[(바람, Noun), (세기, Noun), (조절, Noun), (도, Josa...","[[(단계, Noun), (조절, Noun), (이, Josa), (안되는, Adj..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,[바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아...,[괴물 토네이도 이런 표현은좀 오바네요],"[[(바람, Noun), (나오는, Verb), (부위, Noun), (도, Jos...","[[(괴물, Noun), (토네이도, Noun), (이런, Adjective), (..."
3,온도,냉풍,#차가운 바람 #찬바람,"['냉풍', '차가운 바람', '찬바람', '바람 차갑다']",2476,"[가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요, ...","[냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이 별...","[[(가장, Noun), (큰, Verb), (장점, Noun), (은, Josa)...","[[(냉풍, Noun), (은, Josa), (더운, Adjective), (바람,..."
4,온도,온풍,#따뜻한 바람 #부드러운 바람,"['온풍', '따뜻하다 바람', '부드럽다 바람']",2529,"[파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 바람 적...","[온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로 좋...","[[(파워, Noun), (온, Noun), (풍, Adverb)], [(따뜻한, ...","[[(온풍, Noun), (으로는, Josa), (두피, Noun), (나, Jos..."


In [58]:
data_pos.loc[0,"tag_pos"][0]

[('바람', 'Noun'), ('도', 'Josa'), ('쎄고', 'Adjective')]

In [88]:
df_ = data_pos.copy()
Word = []
for i in range(len(df_)):
    words = df_.loc[i,"tag_pos"]
    words_2 = [w2[1]  for w in words for w2 in w]
    Word.extend(words_2)
#Word   
Counter(Word)

Counter({'Noun': 797,
         'Josa': 321,
         'Adjective': 370,
         'Verb': 232,
         'Adverb': 43,
         'Number': 9,
         'Alpha': 15,
         'Suffix': 10,
         'Modifier': 11,
         'VerbPrefix': 6,
         'Foreign': 1,
         'Determiner': 3,
         'KoreanParticle': 2,
         'Exclamation': 1,
         'Punctuation': 1})

In [107]:
df_ = data_pos.copy()
tar_col = "tag_pos"
key_word = "바람"
def word_vec(df_,tar_col,key_word,version):
    Word = []
    for i in range(len(df_)):
        words = df_.loc[i,tar_col]
        if version == "all":
            words_2 = [w2[0]  for w in words for w2 in w ]
        elif version == "normal":
            words_2 = [w2[0]  for w in words for w2 in w if w2[1]=="Noun" or "Adjective"]
        elif version == "only":
            words_2 = [w2[0]  for w in words for w2 in w if w2[1]==key_word or "Adjective" or "Verb" or "Adverb" ]
        #Word.extend(words_2)
        Word.append(words_2)
    return Word

In [123]:
for i in range(len(df_)):
    words = df_.loc[i,tar_col]
    words_2 = [w2[0]  for w in words for w2 in w ]
    print(words_2)

['바람', '도', '쎄고', '드라이기', '바람', '세기', '만족해요', '바람', '도', '세서', '좋아요', '바람', '세기', '도', '좋아서', '바람', '세기', '도', '맘에듭니', '다', '바람', '세기', '도', '강해서', '강한', '바람', '으로', '젖은', '머리', '가', '금방', '말', '라서', '좋아요', '바람', '이', '세서', '빨리', '말라요', '사용', '하는데', '적당한', '바람세기', '입니다', '바람', '세기', '도', '충분하다', '바람', '세기', '도', '단계', '별로', '있고', '2000', 'w', '라', '바람', '이', '쎄고', '좋네요', '바람', '세기', '는', '최고', '입니다', '바람', '도', '세서', '빨리', '건조', '되니', '만족합니다', '바람', '이', '쎄서', '물기', '도', '금방', '마르는', '느낌', '이', '예요', '바람', '세기', '도', '흡족합니다', '바람', '쎄기', '따라오기', '힘들껍니', '다', '바람', '은', '최상급', '강력한', '바람세기', '입니다', '빠르고', '확실하게', '건조', '가', '가능합니다', '바람', '세기', '가', '엄청나네요', '빠른', '건조', '에', '만족합니다', '고', '출력', '이라서', '그런지', '바람', '완전', '쎄요', '파워', '풀', '한', '바람', '좋아용', '바람', '세기', '최고', '진짜', '속', '시원하게', '나오네요', '바람', '세기', '장난', '아님', '쩌렁', '바람', '세기', '는', '더할', '나위', '없구요', '바람', '세기', '작살', '나', '여', '역시', '전', '문', '가용', '이라', '바람', '세기', '가', '다르네요', '가정', '용', '보단', '확실히', '쎄고', '미용실', '정도', '

In [121]:
for i in range(len(df_)):
    words = df_.loc[i,tar_col]
    words_2 = [w2[0]  for w in words for w2 in w if w2[1]==  "Noun"]
    print(words_2)

['바람', '드라이기', '바람', '세기', '바람', '바람', '세기', '바람', '세기', '맘에듭니', '바람', '세기', '바람', '머리', '금방', '말', '바람', '사용', '바람', '세기', '바람', '세기', '단계', '별로', '바람', '바람', '세기', '최고', '바람', '건조', '바람', '물기', '금방', '느낌', '바람', '세기', '바람', '힘들껍니', '바람', '최상급', '건조', '바람', '세기', '건조', '출력', '바람', '완전', '파워', '풀', '바람', '바람', '세기', '최고', '진짜', '속', '바람', '세기', '장난', '쩌렁', '바람', '세기', '나위', '바람', '세기', '작살', '여', '역시', '가용', '바람', '세기', '가정', '용', '미용실', '정도', '바람', '바람', '세기', '남다름']
['바람', '세기', '조절', '강약', '조절', '단계', '버튼', '조절', '바람', '세기', '조절', '풍량', '조절', '기도', '바람', '세기', '조절', '여러가지', '사용', '정말', '바람', '세기', '세분', '매우', '강도', '조절', '풍량', '조절', '좀', '더', '드라이', '단계', '디테', '일해', '머리', '드라이', '때', '좀더', '것', '단계', '별로', '바람', '세기', '조절', '세기', '채', '바람', '조절', '가능', '바람', '세기', '단', '조절', '가능', '바람', '세기', '조절', '바람', '세기', '완전', '맘']
['바람', '부위', '사용', '때', '내', '더', '토네이도', '바람', '토네이도', '두피', '싹싹']
['가장', '장점', '냉풍', '냉풍', '머릿결', '상하', '냉', '하나', '여름', '냉', '냉', '때문', '여름', '냉풍', '아주', '냉풍', 

In [122]:
for i in range(len(df_)):
    words = df_.loc[i,tar_col]
    words_2 = [w2[0]  for w in words for w2 in w if w2[1]==  "Adjective"]
    print(words_2)

['쎄고', '만족해요', '좋아요', '좋아서', '강해서', '강한', '좋아요', '말라요', '적당한', '입니다', '충분하다', '있고', '쎄고', '좋네요', '입니다', '만족합니다', '쎄서', '마르는', '흡족합니다', '쎄기', '강력한', '입니다', '빠르고', '확실하게', '가능합니다', '엄청나네요', '빠른', '만족합니다', '그런지', '쎄요', '좋아용', '시원하게', '아님', '더할', '없구요', '다르네요', '확실히', '쎄고']
['가능하고요', '같습니다', '가능하고', '가능하니', '좋고', '편합니다', '있어서', '편리하고', '좋습니다', '가능하네요', '좋은', '같아요', '좋다', '섬세하게', '간단하네요']
['편한대로', '좋아여', '같은', '만족합니다']
['같고', '좋네요', '끝내주네요', '덥지', '시원하고', '미지근하지', '깔끔한', '확실해서', '좋다', '뚜렷해서', '좋다', '있어', '좋아요', '편하고', '유용하네요', '차가운', '좋아요', '확실하다', '확실히', '차가', '다릅니다', '시원하고']
['따뜻한', '좋네요', '따뜻한', '따뜻한', '적당합니다', '따뜻한', '상당히', '높네요', '따뜻한', '만족합니다', '따뜻한', '있어서', '좋음', '따뜻한', '따뜻한', '뜨겁지', '좋으네요', '뜨겁지', '따뜻해서', '없다']
['뜨거운', '만족합니다', '뜨겁게', '뜨거운', '뜨겁지', '좋은', '같아요', '뜨겁지', '좋아요']
['확실해서', '좋구요']
['편리하네요']
['좋구요', '있어', '편합니다', '뜨거운', '차가운', '있어서', '편해요']
[]
['가능해요', '가벼워서', '좋아', '있어서', '좋아요', '가능해요', '다양하고', '편해요', '있어요', '가능해요', '좋고요', '좋네요', '좋아요', '있는게', '좋아요', '유용하다', '다양하고', '만족합니다

In [108]:
word_1 = word_vec(df_,"tag_pos","바람","all")
word_2 = word_vec(df_,"tag_pos","바람","normal")
word_3 = word_vec(df_,"tag_pos","바람","only")

In [109]:
TA = word_1
model_1 = Word2Vec(sentences=TA,window=3,min_count=0,iter=100,sg=1)
model_1.most_similar(key_word,topn=10)

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('좋아서', 0.9621203541755676),
 ('만족해요', 0.9439737796783447),
 ('맘에듭니', 0.9385101199150085),
 ('흡족합니다', 0.926663875579834),
 ('세기', 0.92625892162323),
 ('남다름', 0.9215455651283264),
 ('부드러우면서', 0.921137273311615),
 ('간단하네요', 0.9197307229042053),
 ('세고', 0.9165195226669312),
 ('강한', 0.9152929186820984)]

## word2vec

In [55]:
def word2vec_model(df_all,target_col,win_size):
    model_words = Word2Vec(sentences=df_all[target_col], window = win_size, min_count=0,iter=100, sg=1)
    return model_words

In [44]:
model_pos = word2vec_model(data_s,"split_pos",2)